CRUD Method, Create, Read, Update, Delete

Data structures

In [ ]:
#User data structure
 {
  "id": "uuid",
  "name": "string",
  "email": "string",
  "password_hash": "string",
  "houses": ["house_id1", "house_id2"]
}
#House data structure
{
  "id": "uuid",
  "address": "string",
  "owner_id": "uuid",
  "rooms": ["room_id1", "room_id2", "room_id3"]
}
#Room data structure
{
  "id": "uuid",
  "address": "string",
  "owner_id": "uuid",
  "rooms": ["room_id1", "room_id2", "room_id3"]
}

#Device data structure
{
  "id": "uuid",
  "type": "string",
  "room_id": "uuid",
  "status": "on/off",
  "metadata": {}
}


In [ ]:
from flask import Flask, jsonify, request
import uuid

app = Flask(__name__)

# Dummy Data
users = {}


@app.route('/users', methods=['POST'])
def create_user():
    data = request.json
    user_id = str(uuid.uuid4())
    users[user_id] = {
        "id": user_id,
        "name": data.get("name"),
        "email": data.get("email")
    }
    return jsonify(users[user_id]), 201

@app.route('/users/<id>', methods=['GET'])
def get_user(id):
    return jsonify(users.get(id, {"error": "User not found"}))

def home():
    return "Hello, User!"


if __name__ == '__main__':
    app.run(debug=True)


Error handling

In [ ]:
from flask import Flask, request, jsonify
import uuid
import re

app = Flask(__name__)

# Dummy Storage
users = {}
houses = {}
rooms = {}
devices = {}

def is_valid_email(email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email)

@app.route('/users', methods=['POST'])
def create_user():
    data = request.json
    if not data or "name" not in data or "email" not in data:
        return jsonify({"error": "Missing required fields"}), 400
    if not is_valid_email(data["email"]):
        return jsonify({"error": "Invalid email format"}), 400

    user_id = str(uuid.uuid4())
    users[user_id] = {
        "id": user_id,
        "name": data["name"],
        "email": data["email"]
    }
    return jsonify(users[user_id]), 201

@app.route('/users/<id>', methods=['GET'])
def get_user(id):
    if id not in users:
        return jsonify({"error": "User not found"}), 404
    return jsonify(users[id])

@app.route('/users/<id>', methods=['DELETE'])
def delete_user(id):
    if id not in users:
        return jsonify({"error": "User not found"}), 404
    del users[id]
    return jsonify({"message": "User deleted"}), 200

if __name__ == '__main__':
    app.run(debug=True)


Unit Testing

In [ ]:
#Unit Testing
import pytest
from app import app

@pytest.fixture
def client():
    app.config['TESTING'] = True
    with app.test_client() as client:
        yield client

def test_create_user_success(client):
    response = client.post('/users', json={"name": "Alice", "email": "alice@example.com"})
    assert response.status_code == 201

def test_create_user_invalid_email(client):
    response = client.post('/users', json={"name": "Alice", "email": "wrongemail"})
    assert response.status_code == 400
    assert response.get_json() == {"error": "Invalid email format"}

def test_get_user_not_found(client):
    response = client.get('/users/nonexistent_id')
    assert response.status_code == 404
